In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalize(train[:,0])
# without tanh is worse-off

In [3]:
## change ##
batch_size = 16
clip = .5

Prepare data

In [68]:
train_gen = SequenceGen(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGen(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [87]:
convNets = Sequential(ConvBatchLeaky1D(1,2,5,stride=2),
                      ConvBatchLeaky1D(2,4,5,stride=2),
                      ConvBatchLeaky1D(4,8,5,stride=2),
                      ConvBatchLeaky1D(8,16,5,stride=2),
                      ConvBatchLeaky1D(16,32,5,stride=2),
                      DenseBlock(32),
                      DenseBlock(64),
                      Dropout(0.5),
                      DenseBlock(128),
                      Dropout(0.5),
                      DenseBlock(256),
                      Dropout(0.5),
                      GRU_NCL(512,512,batch_first=True,bidirectional=True,returnH=True))

In [88]:
linear = Sequential(Dropout(0.5),Linear(512*2,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))

In [89]:
model = CNN_RNN2seq(convNets,linear).to('cuda:0')

In [90]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [91]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [94]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.803965685459284, val_loss:2.5276477336883545
epoch:1, train_loss:2.502018362856828, val_loss:2.505384922027588
epoch:2, train_loss:2.417132361003986, val_loss:2.4122190475463867
epoch:3, train_loss:2.3946147973720846, val_loss:2.458768844604492
epoch:4, train_loss:2.42960814673167, val_loss:2.409284830093384
Training completed in 151.23462963104248s


Submission

In [102]:
submission = pd.read_csv('../Data/sample_submission.csv',)
test_gen = SequenceGenTest(submission.seg_id.tolist())
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)
submission.iloc[:,1] = predict(model,test_gen)
submission.to_csv('../Submission/sub1.csv',index=False)